# Ejercicio aplicado de DataFrames y Spark SQL

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [1]:
## Importa el csv de "data/WorldCupPlayers.csv"
## Visualiza los datos

In [2]:
## ¿que tipo de datos contiene cada variable?

In [3]:
## ¿Cuantos registros hay?

In [4]:
## Obtén los principales estadísticos de Position

In [5]:
## Slecciona y muestra los registros distintos de 'Player Name','Coach Name'

In [6]:
## ¿Cuantos partidos con el ID de 1096 ha habido?

In [7]:
## Muestra los datos donde la posicion haya sido C y el evento sea G40

In [8]:
## Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20
